In [4]:
import os 
import sys 
import csv 
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.svm import OneClassSVM 
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score 
import tensorflow as tf 
from tensorflow.keras.models import Model, load_model, Sequential 
from tensorflow.keras.layers import Dense, UpSampling2D, Conv2D, Conv2DTranspose, Reshape, Input, BatchNormalization, LeakyReLU, Activation, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.utils import load_img, img_to_array, array_to_img 
from efficientnet.tfkeras import EfficientNetB2, preprocess_input 
from keras.utils.vis_utils import plot_model
import random 


import MVTecAD_loaddata as mv

In [5]:
def train_gan_data(X_train):
    # ハイパーパラメータの設定
    # 潜在変数の次元数
    latent_dim = 1000
    # バッチサイズ
    batch_size = 32
    # エポック数
    epochs = 100
    # 学習率
    lr = 2**(-3)
    # ベータ1
    beta_1 = 0.5

    # Generatorモデルの構築
    def build_generator():
        # Generatorは、潜在変数からRGB画像を生成するモデルです。
        # Conv2DTranspose層とBatchNormalization層とLeakyReLU層を用いて、アップサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Dense(8*8*512, input_dim=latent_dim))
        model.add(Reshape((8, 8, 512)))
        # 中間層1
        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層4
        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        return model

    # Discriminatorモデルの構築
    def build_discriminator():
        # Discriminatorは、RGB画像が本物か偽物かを判定するモデルです。
        # Conv2D層とBatchNormalization層とLeakyReLU層を用いて、ダウンサンプリングを行います。
        model = Sequential()
        # 入力層
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(256, 256 ,3)))
        model.add(LeakyReLU(alpha=0.2))
        # 中間層1
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層2
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 中間層3
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # 出力層
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        return model

    # Generatorモデルの作成
    generator = build_generator()
    # Discriminatorモデルの作成
    discriminator = build_discriminator()
    # Discriminatorモデルのコンパイル
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr, beta_1), metrics=['accuracy'])
    # Discriminatorモデルの学習を停止
    discriminator.trainable = False

    # GANモデルの構築
    # GANモデルは、GeneratorモデルとDiscriminatorモデルを結合したモデルです。
    # 潜在変数を入力として、Discriminatorモデルの出力を得るようにします。
    gan_input = Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = Model(gan_input, gan_output)
    # GANモデルのコンパイル
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr, beta_1))

    # 学習のループ
    for epoch in range(epochs):
        # バッチごとに学習
        for i in range(0, len(X_train), batch_size):
            # 本物の画像データをバッチサイズ分取得
            real_images = X_train[i:i+batch_size]
            # 偽物の画像データをバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), latent_dim))
            fake_images = generator.predict(latent_vectors)
            # 本物の画像データと偽物の画像データを結合
            images = np.concatenate([real_images, fake_images])
            # 本物の画像データにはラベル1を、偽物の画像データにはラベル0を付ける
            labels = np.concatenate([np.ones((len(real_images), 1)), np.zeros((len(real_images), 1))])
            # ラベルにわずかなノイズを加える
            labels += 0.05 * np.random.random(labels.shape)
            # Discriminatorモデルの学習を再開
            discriminator.trainable = True
            # Discriminatorモデルの学習と評価
            loss, acc = discriminator.train_on_batch(images, labels)
            # Discriminatorモデルの学習を停止
            discriminator.trainable = False
            # 潜在変数をバッチサイズ分生成
            latent_vectors = np.random.normal(0, 1, size=(len(real_images), latent_dim))
            # 偽物の画像データにはラベル1を付ける（騙す）
            misleading_labels = np.ones((len(real_images), 1))
            # GANモデルの学習
            gan.train_on_batch(latent_vectors, misleading_labels)
            gan.predict
        # エポックごとに結果を表示
        print(f'Epoch {epoch}, Loss: {loss}, Accuracy: {acc}')
    imageslatent_vectors = np.random.normal(0, 1, size=(len(X_train), latent_dim))
    fake_images = generator.predict(imageslatent_vectors)
    fake_images = fake_images/np.max(fake_images)*255
    return fake_images, generator

# def make_mask(X):
#     X_dam = X
#     for i in range(len(X_dam)):
#         rand_x = randint(32,224)
#         rand_y = randint(32,224)
#         x1, x2 = rand_x-20, rand_x+20
#         y1, y2 = rand_y-20, rand_y+20
#         X_dam[i,x1:x2, y1:y2, :] = 1.0
#     return X_dam

def mask(name, X):
    for i in range(len(X)):
        img = cv2.imread(name+'/train/good/{0:03}.png'.format(i))
        rand_x = random.randint(60,840)
        rand_y = random.randint(60,840)
        x1, x2 = rand_x-60, rand_x+60
        y1, y2 = rand_y-60, rand_y+60
        img[x1:x2, y1:y2] = 255
        cv2.imwrite('fakes/'+name+'/'+str(i)+'.png',img)
        
def make_mask(name, X, back):
    img = cv2.imread(name+'/train/good/000.png', cv2.IMREAD_GRAYSCALE)
    if back==1:
        img_gray=255-img
#     elif back==2:
#         mask(name, X)
#         for i in range(len(X)):
#             img = cv2.imread(name+'/train/good/{0:03}.png'.format(i))
#             rand_x = random.randint(60,840)
#             rand_y = random.randint(60,840)
#             x1, x2 = rand_x-60, rand_x+60
#             y1, y2 = rand_y-60, rand_y+60
#             img[x1:x2, y1:y2] = 255
#             cv2.imwrite('fakes/'+name+'/'+str(i)+'.png',img)
    else:
        img_gray=img
    contours, hierarchy = cv2.findContours(img_gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    max_area = 0
    max_index = -1 
    for i, contour in enumerate(contours): 
        area = cv2.contourArea(contour) 
        if area > max_area: 
            max_area = area
            max_index = i
    max_contour = contours[max_index]
    img_contour = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(img_contour, [max_contour], -1, (0, 255, 0), -1)
    nonzero = np.where(img_contour[:, :, 1]!=0) 
    x = nonzero[0]
    y = nonzero[1]

    for i in range(len(X)):
        img = cv2.imread(name+'/train/good/{0:03}.png'.format(i))
        rand_x = random.choice(x)
        rand_y = random.choice(y)
        x1, x2 = rand_x-60, rand_x+60
        y1, y2 = rand_y-60, rand_y+60
        img[x1:x2, y1:y2] = 255
        cv2.imwrite('fakes/'+name+'/'+str(i)+'.png',img)
    
# def save_image_bottle(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/bottle/'+str(i)+'.png', fake[i])

# def save_image_cable(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/cable/'+str(i)+'.png', fake[i])

# def save_image_capsule(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/capsule/'+str(i)+'.png', fake[i])

# def save_image_hazelnut(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/hazelnut/'+str(i)+'.png', fake[i])

# def save_image_metal_nut(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/metal-nut/'+str(i)+'.png', fake[i])

# def save_image_pill(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/pill/'+str(i)+'.png', fake[i])

# def save_image_screw(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/screw/'+str(i)+'.png', fake[i])

# def save_image_toothbrush(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/toothbrush/'+str(i)+'.png', fake[i])
        
# def save_image_transistor(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/transistor/'+str(i)+'.png', fake[i])

# def save_image_zipper(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('fakes/zipper/'+str(i)+'.png', fake[i]) 

# def save_image_bottle(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/bottle/'+str(i)+'.png', fake[i])

# def save_image_cable(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/cable/'+str(i)+'.png', fake[i])

# def save_image_capsule(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/capsule/'+str(i)+'.png', fake[i])

# def save_image_hazelnut(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/hazelnut/'+str(i)+'.png', fake[i])

# def save_image_metal_nut(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/metal-nut/'+str(i)+'.png', fake[i])

# def save_image_pill(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/pill/'+str(i)+'.png', fake[i])

# def save_image_screw(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/screw/'+str(i)+'.png', fake[i])

# def save_image_toothbrush(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/toothbrush/'+str(i)+'.png', fake[i])
        
# def save_image_transistor(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/transistor/'+str(i)+'.png', fake[i])

# def save_image_zipper(fake):
#     for i in range(len(fake)):
#         cv2.imwrite('mask/zipper/'+str(i)+'.png', fake[i]) 

In [30]:
def mask(X):
    
    X_list = []
    for i in range(len(X)):
        rand_x = random.randint(64,192)
        rand_y = random.randint(84,162)
        x1, x2 = rand_x-15, rand_x+15
        y1, y2 = rand_y-15, rand_y+15
        X[i,x1:x2, y1:y2, :] = 1.0
        X_list.append(X[i,:,:,:])
    X_list = np.array(X_list)*255
    return X_list

def save_mask(img, name):
    for i in range(len(img)):
        cv2.imwrite('fakes/'+name+'/'+str(i)+'.png', img[i])

In [31]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.pill()
list_X = mask(X_train)
save_mask(list_X, 'pill')

In [34]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
make_mask('bottle', X_train, 1)

In [35]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.cable()
make_mask('cable', X_train, 1)

In [36]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.capsule()
make_mask('capsule', X_train, 1)

In [46]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.hazelnut()
make_mask('hazelnut', X_train, 1)

In [47]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.metal_nut()
make_mask('metal_nut', X_train, 1)

In [39]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.pill()
make_mask('pill', X_train, 1)

In [40]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.screw()
make_mask('screw', X_train, 1)

In [48]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.toothbrush()
make_mask('toothbrush', X_train, 1)

In [42]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.transistor()
make_mask('transistor', X_train, 1)

In [44]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.zipper()
make_mask('zipper', X_train, 1)

In [52]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.carpet()
mask('carpet', X_train)

In [53]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.grid()
mask('grid', X_train)

In [54]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.leather()
mask('leather', X_train)

In [55]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.tile()
mask('tile', X_train)

In [56]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.wood()
mask('wood', X_train)

In [18]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_bottle(fake_train)

1/1 [==============================] - 0s 79ms/step
Epoch 0, Loss: -0.3555949628353119, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 1, Loss: -0.09260944277048111, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 2, Loss: 0.13157254457473755, Accuracy: 0.0
1/1 [==============================] - 0s 77ms/step
Epoch 3, Loss: -0.10567736625671387, Accuracy: 0.0
1/1 [==============================] - 0s 71ms/step
Epoch 4, Loss: -0.5711541771888733, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 5, Loss: -0.6658253073692322, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 6, Loss: -0.5817831158638, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 7, Loss: -0.32037240266799927, Accuracy: 0.0
1/1 [==============================] - 0s 76ms/step
Epoch 8, Loss: -0.4117138981819153, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 9, Loss: -0.073377370834

1/1 [==============================] - 0s 70ms/step
Epoch 20, Loss: -2.484017848968506, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 21, Loss: -0.08559492230415344, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 22, Loss: 0.02124476432800293, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 23, Loss: -0.5021774172782898, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 24, Loss: -0.8268247842788696, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 25, Loss: -0.15333479642868042, Accuracy: 0.0
1/1 [==============================] - 0s 72ms/step
Epoch 26, Loss: -1.0097992420196533, Accuracy: 0.0
1/1 [==============================] - 0s 57ms/step
Epoch 27, Loss: -0.6791990995407104, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 28, Loss: -0.19643916189670563, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 29, Loss: -1.

1/1 [==============================] - 0s 76ms/step
Epoch 41, Loss: -2.924376964569092, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 42, Loss: -2.0758492946624756, Accuracy: 0.0
1/1 [==============================] - 0s 67ms/step
Epoch 43, Loss: -5.721410274505615, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 44, Loss: -6.236440181732178, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 45, Loss: -4.705511569976807, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 46, Loss: -5.341209411621094, Accuracy: 0.0
1/1 [==============================] - 0s 66ms/step
Epoch 47, Loss: -6.659144878387451, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 48, Loss: -8.764836311340332, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 49, Loss: -7.512134075164795, Accuracy: 0.0
1/1 [==============================] - 0s 60ms/step
Epoch 50, Loss: -2.0996108055

1/1 [==============================] - 0s 79ms/step
Epoch 62, Loss: -5.766159534454346, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 63, Loss: -2.0366203784942627, Accuracy: 0.0
1/1 [==============================] - 0s 57ms/step
Epoch 64, Loss: -11.83757209777832, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 65, Loss: -12.521429061889648, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 66, Loss: -16.726600646972656, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 67, Loss: -8.328819274902344, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 68, Loss: -0.6799755096435547, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 69, Loss: -8.722138404846191, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 70, Loss: -17.21883773803711, Accuracy: 0.0
1/1 [==============================] - 0s 71ms/step
Epoch 71, Loss: -18.108757

1/1 [==============================] - 0s 63ms/step
Epoch 82, Loss: -23.44024085998535, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 83, Loss: -22.342281341552734, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 84, Loss: -26.950407028198242, Accuracy: 0.0
1/1 [==============================] - 0s 73ms/step
Epoch 85, Loss: -25.48891830444336, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 86, Loss: -11.199458122253418, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 87, Loss: -26.982337951660156, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 88, Loss: -18.55499267578125, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 89, Loss: -38.459842681884766, Accuracy: 0.0
1/1 [==============================] - 0s 73ms/step
Epoch 90, Loss: -10.945740699768066, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 91, Loss: -18.7060

...

1/1 [==============================] - 0s 64ms/step
Epoch 899, Loss: -6072.888671875, Accuracy: 0.0
1/1 [==============================] - 0s 65ms/step
Epoch 900, Loss: -5028.9541015625, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 901, Loss: -4333.43896484375, Accuracy: 0.0
1/1 [==============================] - 0s 67ms/step
Epoch 902, Loss: -3650.79052734375, Accuracy: 0.0
1/1 [==============================] - 0s 64ms/step
Epoch 903, Loss: -4513.0576171875, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 904, Loss: -13768.263671875, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 905, Loss: -3224.84228515625, Accuracy: 0.0
1/1 [==============================] - 0s 66ms/step
Epoch 906, Loss: -5490.8837890625, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 907, Loss: -12715.26171875, Accuracy: 0.0
1/1 [==============================] - 0s 56ms/step
Epoch 908, Loss: -7804.82177734375, Ac

1/1 [==============================] - 0s 63ms/step
Epoch 919, Loss: -10562.8095703125, Accuracy: 0.0
1/1 [==============================] - 0s 66ms/step
Epoch 920, Loss: -14.413047790527344, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 921, Loss: -5741.9150390625, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 922, Loss: -11846.5478515625, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 923, Loss: -6838.5322265625, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 924, Loss: -9786.45703125, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 925, Loss: -12102.1962890625, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 926, Loss: -11696.064453125, Accuracy: 0.0
1/1 [==============================] - 0s 75ms/step
Epoch 927, Loss: -14447.6689453125, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 928, Loss: -322.904479980468

1/1 [==============================] - 0s 63ms/step
Epoch 940, Loss: -12163.4287109375, Accuracy: 0.0
1/1 [==============================] - 0s 60ms/step
Epoch 941, Loss: -941.220458984375, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 942, Loss: -13791.5908203125, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 943, Loss: -9137.837890625, Accuracy: 0.0
1/1 [==============================] - 0s 78ms/step
Epoch 944, Loss: -7548.3095703125, Accuracy: 0.0
1/1 [==============================] - 0s 54ms/step
Epoch 945, Loss: -14710.3916015625, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 946, Loss: -10375.3349609375, Accuracy: 0.0
1/1 [==============================] - 0s 69ms/step
Epoch 947, Loss: -812.9078979492188, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 948, Loss: -10846.7109375, Accuracy: 0.0
1/1 [==============================] - 0s 65ms/step
Epoch 949, Loss: -2348.63037109375,

1/1 [==============================] - 0s 64ms/step
Epoch 961, Loss: -4093.477294921875, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 962, Loss: -3966.94287109375, Accuracy: 0.0
1/1 [==============================] - 0s 51ms/step
Epoch 963, Loss: -6704.083984375, Accuracy: 0.0
1/1 [==============================] - 0s 56ms/step
Epoch 964, Loss: -5772.333984375, Accuracy: 0.0
1/1 [==============================] - 0s 80ms/step
Epoch 965, Loss: -1922.514404296875, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 966, Loss: -5296.67626953125, Accuracy: 0.0
1/1 [==============================] - 0s 71ms/step
Epoch 967, Loss: -709.8505249023438, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 968, Loss: -3771.520751953125, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 969, Loss: -6748.1220703125, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 970, Loss: -805.266052246

1/1 [==============================] - 0s 63ms/step
Epoch 981, Loss: -1846.9693603515625, Accuracy: 0.0
1/1 [==============================] - 0s 64ms/step
Epoch 982, Loss: -4696.478515625, Accuracy: 0.0
1/1 [==============================] - 0s 79ms/step
Epoch 983, Loss: -10462.6640625, Accuracy: 0.0
1/1 [==============================] - 0s 76ms/step
Epoch 984, Loss: -497.0873107910156, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 985, Loss: -11566.052734375, Accuracy: 0.0
1/1 [==============================] - 0s 60ms/step
Epoch 986, Loss: -13340.3427734375, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 987, Loss: -1205.5484619140625, Accuracy: 0.0
1/1 [==============================] - 0s 82ms/step
Epoch 988, Loss: -14728.5546875, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 989, Loss: -10412.3212890625, Accuracy: 0.0
1/1 [==============================] - 0s 63ms/step
Epoch 990, Loss: -3239.89721679687

In [7]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.cable()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_cable(fake_train)

1/1 [==============================] - 0s 453ms/step
Epoch 0, Loss: 2.035147872447851e-08, Accuracy: 1.0
1/1 [==============================] - 0s 451ms/step
Epoch 1, Loss: 2.070941462761766e-08, Accuracy: 1.0
1/1 [==============================] - 0s 437ms/step
Epoch 2, Loss: 2.1047267040330553e-08, Accuracy: 1.0
1/1 [==============================] - 0s 475ms/step
Epoch 3, Loss: 2.1680470752016845e-08, Accuracy: 1.0
1/1 [==============================] - 0s 492ms/step
Epoch 4, Loss: 2.2462989690552604e-08, Accuracy: 1.0
1/1 [==============================] - 0s 453ms/step
Epoch 5, Loss: 2.327608328300812e-08, Accuracy: 1.0
1/1 [==============================] - 0s 468ms/step
Epoch 6, Loss: 2.4429706257933503e-08, Accuracy: 1.0
1/1 [==============================] - 0s 454ms/step
Epoch 7, Loss: 2.595683312733854e-08, Accuracy: 1.0
1/1 [==============================] - 0s 453ms/step
Epoch 8, Loss: 2.7339670083392775e-08, Accuracy: 1.0
1/1 [==============================] - 0s 454ms/st

1/1 [==============================] - 0s 438ms/step
Epoch 20, Loss: 1.6080947773389198e-07, Accuracy: 1.0
1/1 [==============================] - 0s 454ms/step
Epoch 21, Loss: 5.460019281144923e-08, Accuracy: 1.0
1/1 [==============================] - 0s 439ms/step
Epoch 22, Loss: 1.130314188912962e-07, Accuracy: 1.0
1/1 [==============================] - 1s 505ms/step
Epoch 23, Loss: 2.3744939880998572e-06, Accuracy: 1.0
1/1 [==============================] - 0s 454ms/step
Epoch 24, Loss: 1.0918054016428869e-08, Accuracy: 1.0
1/1 [==============================] - 0s 451ms/step
Epoch 25, Loss: 3.4684624061753766e-09, Accuracy: 1.0
1/1 [==============================] - 0s 454ms/step
Epoch 26, Loss: 5.9875713276369424e-09, Accuracy: 1.0
1/1 [==============================] - 0s 463ms/step
Epoch 27, Loss: 26.309680938720703, Accuracy: 0.46875
1/1 [==============================] - 0s 454ms/step
Epoch 28, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 456ms/step
Epoch

1/1 [==============================] - 0s 460ms/step
Epoch 41, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 466ms/step
Epoch 42, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 466ms/step
Epoch 43, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 470ms/step
Epoch 44, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 455ms/step
Epoch 45, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 481ms/step
Epoch 46, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 453ms/step
Epoch 47, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 470ms/step
Epoch 48, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 453ms/step
Epoch 49, Loss: 0.0, Accuracy: 1.0
7/7 [==============================] - 6s 829ms/step


In [9]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.capsule()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_capsule(fake_train)

1/1 [==============================] - 0s 310ms/step
Epoch 0, Loss: 1.8555825279098492e-11, Accuracy: 1.0
1/1 [==============================] - 0s 312ms/step
Epoch 1, Loss: 2.0532507985504544e-11, Accuracy: 1.0
1/1 [==============================] - 0s 312ms/step
Epoch 2, Loss: 2.4650947949567126e-11, Accuracy: 1.0
1/1 [==============================] - 0s 317ms/step
Epoch 3, Loss: 3.08728112519141e-11, Accuracy: 1.0
1/1 [==============================] - 0s 310ms/step
Epoch 4, Loss: 4.009642270985658e-11, Accuracy: 1.0
1/1 [==============================] - 0s 317ms/step
Epoch 5, Loss: 5.887176857122256e-11, Accuracy: 1.0
1/1 [==============================] - 0s 316ms/step
Epoch 6, Loss: 1.0105982717334427e-10, Accuracy: 1.0
1/1 [==============================] - 0s 314ms/step
Epoch 7, Loss: 1.9656622163299886e-10, Accuracy: 1.0
1/1 [==============================] - 0s 313ms/step
Epoch 8, Loss: 4.325334534982517e-10, Accuracy: 1.0
1/1 [==============================] - 0s 307ms/ste

1/1 [==============================] - 0s 301ms/step
Epoch 20, Loss: 5.0246566019040984e-08, Accuracy: 1.0
1/1 [==============================] - 0s 309ms/step
Epoch 21, Loss: 5.020914528586218e-08, Accuracy: 1.0
1/1 [==============================] - 0s 304ms/step
Epoch 22, Loss: 5.4064980048451616e-08, Accuracy: 1.0
1/1 [==============================] - 0s 329ms/step
Epoch 23, Loss: 4.931735375635071e-08, Accuracy: 1.0
1/1 [==============================] - 0s 317ms/step
Epoch 24, Loss: 3.5984957236223636e-08, Accuracy: 1.0
1/1 [==============================] - 0s 312ms/step
Epoch 25, Loss: 3.59783776104905e-08, Accuracy: 1.0
1/1 [==============================] - 0s 308ms/step
Epoch 26, Loss: 3.051277630561344e-08, Accuracy: 1.0
1/1 [==============================] - 0s 308ms/step
Epoch 27, Loss: 3.151076555241161e-08, Accuracy: 1.0
1/1 [==============================] - 0s 334ms/step
Epoch 28, Loss: 4.814737764036181e-08, Accuracy: 1.0
1/1 [==============================] - 0s 31

1/1 [==============================] - 0s 316ms/step
Epoch 41, Loss: 6.283341599240935e-36, Accuracy: 1.0
1/1 [==============================] - 0s 320ms/step
Epoch 42, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 310ms/step
Epoch 43, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 44, Loss: 3.2554674476208244e-21, Accuracy: 1.0
1/1 [==============================] - 0s 321ms/step
Epoch 45, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 324ms/step
Epoch 46, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 47, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 326ms/step
Epoch 48, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 49, Loss: 0.0, Accuracy: 1.0
7/7 [==============================] - 6s 794ms/step


In [10]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.hazelnut()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_hazelnut(fake_train)

1/1 [==============================] - 0s 208ms/step
Epoch 0, Loss: 1.4839282691525568e-08, Accuracy: 1.0
1/1 [==============================] - 0s 229ms/step
Epoch 1, Loss: 1.7118768624868608e-08, Accuracy: 1.0
1/1 [==============================] - 0s 218ms/step
Epoch 2, Loss: 2.0783696541570862e-08, Accuracy: 1.0
1/1 [==============================] - 0s 212ms/step
Epoch 3, Loss: 2.5066645648053054e-08, Accuracy: 1.0
1/1 [==============================] - 0s 209ms/step
Epoch 4, Loss: 3.096365475130369e-08, Accuracy: 1.0
1/1 [==============================] - 0s 209ms/step
Epoch 5, Loss: 3.682246330072303e-08, Accuracy: 1.0
1/1 [==============================] - 0s 231ms/step
Epoch 6, Loss: 3.922516711440949e-08, Accuracy: 1.0
1/1 [==============================] - 0s 219ms/step
Epoch 7, Loss: 4.3755765233299826e-08, Accuracy: 1.0
1/1 [==============================] - 0s 225ms/step
Epoch 8, Loss: 3.9855212463635326e-08, Accuracy: 1.0
1/1 [==============================] - 0s 214ms/s

1/1 [==============================] - 0s 198ms/step
Epoch 11, Loss: 4.508066808739386e-08, Accuracy: 1.0
1/1 [==============================] - 0s 207ms/step
Epoch 12, Loss: 3.840750495420764e-10, Accuracy: 1.0
1/1 [==============================] - 0s 210ms/step
Epoch 13, Loss: 3.722757102586627e-10, Accuracy: 1.0
1/1 [==============================] - 0s 208ms/step
Epoch 14, Loss: 2.9434108483883392e-09, Accuracy: 1.0
1/1 [==============================] - 0s 226ms/step
Epoch 15, Loss: 9.829778102599107e-10, Accuracy: 1.0
1/1 [==============================] - 0s 221ms/step
Epoch 16, Loss: 1.1358705087616272e-08, Accuracy: 1.0
1/1 [==============================] - 0s 206ms/step
Epoch 17, Loss: 5.24966825565798e-09, Accuracy: 1.0
1/1 [==============================] - 0s 222ms/step
Epoch 18, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 220ms/step
Epoch 19, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 233ms/step
Epoch 20, Loss: 0.0, Accurac

1/1 [==============================] - 0s 215ms/step
Epoch 23, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 201ms/step
Epoch 24, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 218ms/step
Epoch 25, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 211ms/step
Epoch 26, Loss: 4.473088264465332, Accuracy: 0.9285714030265808
1/1 [==============================] - 0s 206ms/step
Epoch 27, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 223ms/step
Epoch 28, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 212ms/step
Epoch 29, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 224ms/step
Epoch 30, Loss: 1.2208052910805046e-22, Accuracy: 1.0
1/1 [==============================] - 0s 206ms/step
Epoch 31, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 206ms/step
Epoch 32, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 219ms/step
Epoch 33, Loss: 0.0

1/1 [==============================] - 0s 211ms/step
Epoch 35, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 224ms/step
Epoch 36, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 209ms/step
Epoch 37, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 224ms/step
Epoch 38, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 211ms/step
Epoch 39, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 213ms/step
Epoch 40, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 222ms/step
Epoch 41, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 213ms/step
Epoch 42, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 206ms/step
Epoch 43, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 229ms/step
Epoch 44, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 215ms/step
Epoch 45, Loss: 0.0, Accuracy: 1.0
1/1 [===========================

1/1 [==============================] - 0s 225ms/step
Epoch 47, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 217ms/step
Epoch 48, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 201ms/step
Epoch 49, Loss: 0.0, Accuracy: 1.0
13/13 [==============================] - 10s 755ms/step


In [11]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.metal_nut()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_metal_nut(fake_train)

1/1 [==============================] - 0s 341ms/step
Epoch 0, Loss: 7.405848805319692e-07, Accuracy: 1.0
1/1 [==============================] - 0s 357ms/step
Epoch 1, Loss: 3.5837197742694116e-07, Accuracy: 1.0
1/1 [==============================] - 0s 353ms/step
Epoch 2, Loss: 2.371873506490374e-07, Accuracy: 1.0
1/1 [==============================] - 0s 338ms/step
Epoch 3, Loss: 1.835206830946845e-07, Accuracy: 1.0
1/1 [==============================] - 0s 354ms/step
Epoch 4, Loss: 1.477527575843851e-07, Accuracy: 1.0
1/1 [==============================] - 0s 350ms/step
Epoch 5, Loss: 1.277595060855674e-07, Accuracy: 1.0
1/1 [==============================] - 0s 340ms/step
Epoch 6, Loss: 1.1363551521981208e-07, Accuracy: 1.0
1/1 [==============================] - 0s 336ms/step
Epoch 7, Loss: 1.0401060990261612e-07, Accuracy: 1.0
1/1 [==============================] - 0s 335ms/step
Epoch 8, Loss: 9.774430509423837e-08, Accuracy: 1.0
1/1 [==============================] - 0s 368ms/step

1/1 [==============================] - 0s 352ms/step
Epoch 20, Loss: 7.124288714521754e-09, Accuracy: 1.0
1/1 [==============================] - 0s 348ms/step
Epoch 21, Loss: 1.698163387686691e-08, Accuracy: 1.0
1/1 [==============================] - 0s 325ms/step
Epoch 22, Loss: 2.2065984595087684e-08, Accuracy: 1.0
1/1 [==============================] - 0s 339ms/step
Epoch 23, Loss: 2.2167654378790758e-08, Accuracy: 1.0
1/1 [==============================] - 0s 340ms/step
Epoch 24, Loss: 1.7459053092139243e-09, Accuracy: 1.0
1/1 [==============================] - 0s 321ms/step
Epoch 25, Loss: 5.015141635311693e-09, Accuracy: 1.0
1/1 [==============================] - 0s 341ms/step
Epoch 26, Loss: 5.5959517020198746e-09, Accuracy: 1.0
1/1 [==============================] - 0s 333ms/step
Epoch 27, Loss: 4.307497025735074e-09, Accuracy: 1.0
1/1 [==============================] - 0s 345ms/step
Epoch 28, Loss: 1.165336604991296e-09, Accuracy: 1.0
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 351ms/step
Epoch 41, Loss: 1.4381150608185924e-29, Accuracy: 1.0
1/1 [==============================] - 0s 335ms/step
Epoch 42, Loss: 6.211847938191828e-16, Accuracy: 1.0
1/1 [==============================] - 0s 351ms/step
Epoch 43, Loss: 1.8636129480883154e-33, Accuracy: 1.0
1/1 [==============================] - 0s 349ms/step
Epoch 44, Loss: 6.773908826441036e-14, Accuracy: 1.0
1/1 [==============================] - 0s 341ms/step
Epoch 45, Loss: 3.1346218838097715e-25, Accuracy: 1.0
1/1 [==============================] - 0s 343ms/step
Epoch 46, Loss: 1.1307035368272142e-17, Accuracy: 1.0
1/1 [==============================] - 0s 351ms/step
Epoch 47, Loss: 2.2761990482403467e-17, Accuracy: 1.0
1/1 [==============================] - 0s 341ms/step
Epoch 48, Loss: 3.226465625987283e-31, Accuracy: 1.0
1/1 [==============================] - 0s 341ms/step
Epoch 49, Loss: 1.015677072918389e-28, Accuracy: 1.0
7/7 [==============================] - 6s

In [12]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.pill()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_pill(fake_train)

1/1 [==============================] - 0s 312ms/step
Epoch 0, Loss: 4.401805142606463e-07, Accuracy: 1.0
1/1 [==============================] - 0s 342ms/step
Epoch 1, Loss: 2.2348473294186988e-07, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 2, Loss: 1.8277472690897412e-07, Accuracy: 1.0
1/1 [==============================] - 0s 309ms/step
Epoch 3, Loss: 1.862096326021856e-07, Accuracy: 1.0
1/1 [==============================] - 0s 309ms/step
Epoch 4, Loss: 1.9867584910571168e-07, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 5, Loss: 1.8082765507188014e-07, Accuracy: 1.0
1/1 [==============================] - 0s 309ms/step
Epoch 6, Loss: 1.5108668094399036e-07, Accuracy: 1.0
1/1 [==============================] - 0s 322ms/step
Epoch 7, Loss: 1.3760508466020838e-07, Accuracy: 1.0
1/1 [==============================] - 0s 324ms/step
Epoch 8, Loss: 1.7011171848935192e-07, Accuracy: 1.0
1/1 [==============================] - 0s 324ms/

1/1 [==============================] - 0s 327ms/step
Epoch 17, Loss: 2.8681090835647183e-09, Accuracy: 1.0
1/1 [==============================] - 0s 327ms/step
Epoch 18, Loss: 1.0999903210517914e-08, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 19, Loss: 5.8572776850951414e-08, Accuracy: 1.0
1/1 [==============================] - 0s 332ms/step
Epoch 20, Loss: 4.881212767315901e-09, Accuracy: 1.0
1/1 [==============================] - 0s 311ms/step
Epoch 21, Loss: 1.6418275627927414e-09, Accuracy: 1.0
1/1 [==============================] - 0s 312ms/step
Epoch 22, Loss: 6.651717399108747e-07, Accuracy: 1.0
1/1 [==============================] - 0s 344ms/step
Epoch 23, Loss: 1.2869297294493265e-10, Accuracy: 1.0
1/1 [==============================] - 0s 351ms/step
Epoch 24, Loss: 3.1381633402993714e-10, Accuracy: 1.0
1/1 [==============================] - 0s 320ms/step
Epoch 25, Loss: 1.1121673026970313e-10, Accuracy: 1.0
1/1 [==============================] - 

1/1 [==============================] - 0s 309ms/step
Epoch 34, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 335ms/step
Epoch 35, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 318ms/step
Epoch 36, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 325ms/step
Epoch 37, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 38, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 320ms/step
Epoch 39, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 317ms/step
Epoch 40, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 323ms/step
Epoch 41, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 339ms/step
Epoch 42, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 322ms/step
Epoch 43, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 319ms/step
Epoch 44, Loss: 0.0, Accuracy: 1.0
1/1 [===========================

In [13]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.screw()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_screw(fake_train)

1/1 [==============================] - 1s 864ms/step
Epoch 0, Loss: 5.333896636962891, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 1, Loss: -12.280725479125977, Accuracy: 0.0
1/1 [==============================] - 1s 926ms/step
Epoch 2, Loss: 29372.654296875, Accuracy: 0.0
1/1 [==============================] - 1s 880ms/step
Epoch 3, Loss: 4.920709133148193, Accuracy: 0.0
1/1 [==============================] - 1s 861ms/step
Epoch 4, Loss: 7.308215141296387, Accuracy: 0.0
1/1 [==============================] - 1s 852ms/step
Epoch 5, Loss: 3.629574775695801, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 6, Loss: 7.130090713500977, Accuracy: 0.0
1/1 [==============================] - 1s 880ms/step
Epoch 7, Loss: 4.40559196472168, Accuracy: 0.0
1/1 [==============================] - 1s 859ms/step
Epoch 8, Loss: 1.747204065322876, Accuracy: 0.0
1/1 [==============================] - 1s 855ms/step
Epoch 9, Loss: 0.8987436294555664, Accuracy: 

1/1 [==============================] - 1s 1s/step
Epoch 29, Loss: 2.9966166019439697, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 30, Loss: -9.242244720458984, Accuracy: 0.0
1/1 [==============================] - 1s 808ms/step
Epoch 31, Loss: -15.217595100402832, Accuracy: 0.0
1/1 [==============================] - 1s 788ms/step
Epoch 32, Loss: -32.94041442871094, Accuracy: 0.0
1/1 [==============================] - 2s 2s/step
Epoch 33, Loss: -60.86130142211914, Accuracy: 0.0
1/1 [==============================] - 2s 2s/step
Epoch 34, Loss: -57.90071105957031, Accuracy: 0.0
1/1 [==============================] - 2s 2s/step
Epoch 35, Loss: -58.78255844116211, Accuracy: 0.0
1/1 [==============================] - 2s 2s/step
Epoch 36, Loss: -101.63322448730469, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 37, Loss: -153.13941955566406, Accuracy: 0.0
1/1 [==============================] - 2s 2s/step
Epoch 38, Loss: 6.955498695373535, Accura

1/1 [==============================] - 1s 911ms/step
Epoch 57, Loss: -1312.1337890625, Accuracy: 0.0
1/1 [==============================] - 1s 996ms/step
Epoch 58, Loss: -926.0780029296875, Accuracy: 0.0
1/1 [==============================] - 1s 881ms/step
Epoch 59, Loss: -1706.734619140625, Accuracy: 0.0
1/1 [==============================] - 1s 902ms/step
Epoch 60, Loss: -1490.1005859375, Accuracy: 0.0
1/1 [==============================] - 1s 877ms/step
Epoch 61, Loss: -2139.19091796875, Accuracy: 0.0
1/1 [==============================] - 1s 896ms/step
Epoch 62, Loss: -2403.13623046875, Accuracy: 0.0
1/1 [==============================] - 1s 853ms/step
Epoch 63, Loss: -4242.72998046875, Accuracy: 0.0
1/1 [==============================] - 1s 847ms/step
Epoch 64, Loss: -2823.58837890625, Accuracy: 0.0
1/1 [==============================] - 1s 917ms/step
Epoch 65, Loss: -4344.9619140625, Accuracy: 0.0
1/1 [==============================] - 1s 839ms/step
Epoch 66, Loss: -5107.4296875,

1/1 [==============================] - 1s 841ms/step
Epoch 72, Loss: -6435.75927734375, Accuracy: 0.0
1/1 [==============================] - 1s 855ms/step
Epoch 73, Loss: -9770.0810546875, Accuracy: 0.0
1/1 [==============================] - 1s 842ms/step
Epoch 74, Loss: -9371.328125, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 75, Loss: -14019.255859375, Accuracy: 0.0
1/1 [==============================] - 2s 2s/step
Epoch 76, Loss: -15229.37109375, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 77, Loss: -13759.64453125, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 78, Loss: -20391.24609375, Accuracy: 0.0
1/1 [==============================] - 1s 854ms/step
Epoch 79, Loss: -18808.2890625, Accuracy: 0.0
1/1 [==============================] - 1s 908ms/step
Epoch 80, Loss: -19804.841796875, Accuracy: 0.0
1/1 [==============================] - 1s 1s/step
Epoch 81, Loss: -24214.515625, Accuracy: 0.0
1/1 [===========

In [14]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.toothbrush()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_toothbrush(fake_train)

1/1 [==============================] - 0s 338ms/step
Epoch 0, Loss: 5.433592377812602e-06, Accuracy: 1.0
1/1 [==============================] - 0s 341ms/step
Epoch 1, Loss: 6.94230436693033e-07, Accuracy: 1.0
1/1 [==============================] - 0s 342ms/step
Epoch 2, Loss: 5.261546220935998e-07, Accuracy: 1.0
1/1 [==============================] - 0s 342ms/step
Epoch 3, Loss: 4.578162986490497e-07, Accuracy: 1.0
1/1 [==============================] - 0s 359ms/step
Epoch 4, Loss: 4.4846467517345445e-07, Accuracy: 1.0
1/1 [==============================] - 0s 346ms/step
Epoch 5, Loss: 4.361305911970703e-07, Accuracy: 1.0
1/1 [==============================] - 0s 355ms/step
Epoch 6, Loss: 4.2602877670105954e-07, Accuracy: 1.0
1/1 [==============================] - 0s 343ms/step
Epoch 7, Loss: 4.5713758822785167e-07, Accuracy: 1.0
1/1 [==============================] - 0s 352ms/step
Epoch 8, Loss: 4.5956235794619715e-07, Accuracy: 1.0
1/1 [==============================] - 0s 362ms/step

In [15]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.transistor()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_transistor(fake_train)

1/1 [==============================] - 0s 159ms/step
Epoch 0, Loss: 1.671192109142794e-07, Accuracy: 1.0
1/1 [==============================] - 0s 171ms/step
Epoch 1, Loss: 1.2129278559314116e-07, Accuracy: 1.0
1/1 [==============================] - 0s 174ms/step
Epoch 2, Loss: 1.0829425889369304e-07, Accuracy: 1.0
1/1 [==============================] - 0s 189ms/step
Epoch 3, Loss: 1.1366478247509804e-07, Accuracy: 1.0
1/1 [==============================] - 0s 162ms/step
Epoch 4, Loss: 1.213573170844029e-07, Accuracy: 1.0
1/1 [==============================] - 0s 162ms/step
Epoch 5, Loss: 1.2455964792934537e-07, Accuracy: 1.0
1/1 [==============================] - 0s 162ms/step
Epoch 6, Loss: 1.2572654384257476e-07, Accuracy: 1.0
1/1 [==============================] - 0s 173ms/step
Epoch 7, Loss: 1.3247461083665257e-07, Accuracy: 1.0
1/1 [==============================] - 0s 161ms/step
Epoch 8, Loss: 1.3134180676388496e-07, Accuracy: 1.0
1/1 [==============================] - 0s 175ms/

1/1 [==============================] - 0s 155ms/step
Epoch 20, Loss: 2.891437134167063e-08, Accuracy: 1.0
1/1 [==============================] - 0s 158ms/step
Epoch 21, Loss: 1.4300101547348731e-08, Accuracy: 1.0
1/1 [==============================] - 0s 183ms/step
Epoch 22, Loss: 7.987573269474524e-08, Accuracy: 1.0
1/1 [==============================] - 0s 150ms/step
Epoch 23, Loss: 1.3120898145757565e-08, Accuracy: 1.0
1/1 [==============================] - 0s 176ms/step
Epoch 24, Loss: 5.078992515450409e-08, Accuracy: 1.0
1/1 [==============================] - 0s 164ms/step
Epoch 25, Loss: 2.362046391812811e-10, Accuracy: 1.0
1/1 [==============================] - 0s 166ms/step
Epoch 26, Loss: 3.2752295320293e-10, Accuracy: 1.0
1/1 [==============================] - 0s 167ms/step
Epoch 27, Loss: 7.930234247055523e-10, Accuracy: 1.0
1/1 [==============================] - 0s 159ms/step
Epoch 28, Loss: 11.796218872070312, Accuracy: 0.5
1/1 [==============================] - 0s 163ms/s

1/1 [==============================] - 0s 159ms/step
Epoch 30, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 175ms/step
Epoch 31, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 175ms/step
Epoch 32, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 174ms/step
Epoch 33, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 170ms/step
Epoch 34, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 174ms/step
Epoch 35, Loss: 9.169327741275354e-35, Accuracy: 1.0
1/1 [==============================] - 0s 167ms/step
Epoch 36, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 168ms/step
Epoch 37, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 157ms/step
Epoch 38, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 152ms/step
Epoch 39, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 158ms/step
Epoch 40, Loss: 0.0, Accuracy: 1.0
1/1 [=========

1/1 [==============================] - 0s 162ms/step
Epoch 41, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 162ms/step
Epoch 42, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 166ms/step
Epoch 43, Loss: 4.981187803864027e-36, Accuracy: 1.0
1/1 [==============================] - 0s 167ms/step
Epoch 44, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 163ms/step
Epoch 45, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 162ms/step
Epoch 46, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 149ms/step
Epoch 47, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 166ms/step
Epoch 48, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 198ms/step
Epoch 49, Loss: 0.0, Accuracy: 1.0
7/7 [==============================] - 5s 760ms/step


In [16]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.zipper()
X_dam = make_mask(X_train)
fake_train, G = train_gan_data(X_dam)
save_image_zipper(fake_train)

1/1 [==============================] - 0s 438ms/step
Epoch 0, Loss: 2.357225309043698e-12, Accuracy: 1.0
1/1 [==============================] - 0s 455ms/step
Epoch 1, Loss: 2.8312638691851966e-12, Accuracy: 1.0
1/1 [==============================] - 0s 454ms/step
Epoch 2, Loss: 3.839877200928488e-12, Accuracy: 1.0
1/1 [==============================] - 0s 461ms/step
Epoch 3, Loss: 5.56841942347952e-12, Accuracy: 1.0
1/1 [==============================] - 0s 485ms/step
Epoch 4, Loss: 8.874491419508246e-12, Accuracy: 1.0
1/1 [==============================] - 0s 450ms/step
Epoch 5, Loss: 1.6139914058022065e-11, Accuracy: 1.0
1/1 [==============================] - 0s 455ms/step
Epoch 6, Loss: 3.791906025285563e-11, Accuracy: 1.0
1/1 [==============================] - 0s 443ms/step
Epoch 7, Loss: 1.1080718287370672e-10, Accuracy: 1.0
1/1 [==============================] - 0s 442ms/step
Epoch 8, Loss: 4.367224915036161e-10, Accuracy: 1.0
1/1 [==============================] - 0s 438ms/step


1/1 [==============================] - 0s 431ms/step
Epoch 19, Loss: 2.1545769612885124e-08, Accuracy: 1.0
1/1 [==============================] - 0s 452ms/step
Epoch 20, Loss: 1.8632333009804825e-08, Accuracy: 1.0
1/1 [==============================] - 0s 434ms/step
Epoch 21, Loss: 3.204280574209406e-08, Accuracy: 1.0
1/1 [==============================] - 0s 438ms/step
Epoch 22, Loss: 1.8057225048551118e-08, Accuracy: 1.0
1/1 [==============================] - 0s 458ms/step
Epoch 23, Loss: 1.127291926472651e-09, Accuracy: 1.0
1/1 [==============================] - 0s 439ms/step
Epoch 24, Loss: 3.273454618479832e-09, Accuracy: 1.0
1/1 [==============================] - 0s 438ms/step
Epoch 25, Loss: 4.4106155172585204e-08, Accuracy: 1.0
1/1 [==============================] - 0s 445ms/step
Epoch 26, Loss: 1.3824290157060659e-08, Accuracy: 1.0
1/1 [==============================] - 0s 455ms/step
Epoch 27, Loss: 8.35659985654047e-09, Accuracy: 1.0
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 450ms/step
Epoch 38, Loss: 8.590081214904785, Accuracy: 0.96875
1/1 [==============================] - 0s 440ms/step
Epoch 39, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 452ms/step
Epoch 40, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 453ms/step
Epoch 41, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 437ms/step
Epoch 42, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 439ms/step
Epoch 43, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 446ms/step
Epoch 44, Loss: 0.0, Accuracy: 1.0
1/1 [==============================] - 0s 460ms/step
Epoch 45, Loss: 7.500964634447361e-32, Accuracy: 1.0
1/1 [==============================] - 0s 449ms/step
Epoch 46, Loss: 7.24542729354188e-29, Accuracy: 1.0
1/1 [==============================] - 0s 453ms/step
Epoch 47, Loss: 1.1524949222276863e-32, Accuracy: 1.0
1/1 [==============================] - 0s 450ms/

In [ ]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.zipper()
X_dam = make_mask(X_train)
# fake_train, G = train_gan_data(X_dam)
save_image_zipper(X_dam)

In [ ]:
X_train, X_test, X_test_good, X_test_error, y_train, y_tg, y_te, y_test = mv.bottle()
X_dam = make_mask(X_train)